# Import Required Libraries
Import the necessary libraries, including pandas, torch, os, tqdm, conllu, sklearn, and transformers.

In [ ]:
import pandas as pd
import torch
import os
import tqdm
from conllu import parse
from torch.utils.data.dataset import Dataset
from sklearn.model_selection import train_test_split
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from torch.utils.data import DataLoader
from torch.optim import AdamW
from sklearn.metrics import hamming_loss, f1_score, classification_report

# Load and Map Labels
Define a function to load and map labels from a given file path.

In [ ]:
def load_and_map_labels(label_file_path):
    labels_df = pd.read_csv(label_file_path, sep="\t", header=None, names=["article_id", "narratives", "subnarratives"])
    
    labels_mapping = {
        row["article_id"]: {
            "narratives": row["narratives"].split(";"),
            "subnarratives": row["subnarratives"].split(";")
        }
        for _, row in labels_df.iterrows()
    }
    
    return labels_mapping

label_file = "../data/training_data_16_October_release/EN/subtask-2-annotations.txt"
labels_mapping = load_and_map_labels(label_file)
print(labels_mapping["EN_UA_015443.txt"])

{'narratives': ['URW: Discrediting the West, Diplomacy', 'URW: Amplifying war-related fears', 'URW: Amplifying war-related fears'], 'subnarratives': ['URW: Discrediting the West, Diplomacy: West is tired of Ukraine', 'URW: Amplifying war-related fears: NATO should/will directly intervene', 'URW: Amplifying war-related fears: By continuing the war we risk WWIII']}


# Parse and Load Articles
Define functions to parse CoNLL-U files and load articles from a given path.

In [ ]:

def parse_conllu_file(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        data = f.read()
    token_lists = parse(data)
    all_tokens = [token["form"] for token_list in token_lists for token in token_list]
    return " ".join(all_tokens)

def load_articles_from_conllu(articles_path, article_ids, labels_mapping):
    articles_data = []
    for article_id in article_ids:
        file_path = os.path.join(articles_path, f"{article_id.replace('.txt', '.conllu')}")
        if os.path.exists(file_path):
            article_text = parse_conllu_file(file_path)
            labels = labels_mapping.get(article_id, {"narratives": [], "subnarratives": []})
            articles_data.append({
                "article_id": article_id,
                "text": article_text,
                "narratives": labels["narratives"],
                "subnarratives": labels["subnarratives"]
            })
    return articles_data

articles_path = "../data/tmp/EN"
article_ids = labels_mapping.keys()
articles_data = load_articles_from_conllu(articles_path, article_ids, labels_mapping)

# Initialize Tokenizer
Initialize the RoBERTa tokenizer using the 'roberta-base' model.

In [ ]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

def tokenize_function(example):
    return tokenizer(
        example["text"],
        padding="max_length",
        truncation=True,
        max_length=512,
        return_tensors="pt"
    )

tokenized_data = [tokenize_function(article) for article in articles_data]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

c:\Users\ahmed\anaconda3\lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ahmed\.cache\huggingface\hub\models--roberta-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

# Tokenize Data
Define a function to tokenize the data using the RoBERTa tokenizer.

In [ ]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

def tokenize_function(example):
    return tokenizer(
        example["text"],
        padding="max_length",
        truncation=True,
        max_length=512,
        return_tensors="pt"
    )

tokenized_data = [tokenize_function(article) for article in articles_data]

# Generate Label Vocabulary
Generate label vocabularies for narratives and subnarratives and create encoding functions.

In [ ]:

all_narratives = sorted({n for article in articles_data for n in article["narratives"]})
all_subnarratives = sorted({sn for article in articles_data for sn in article["subnarratives"]})

narrative_to_index = {n: i for i, n in enumerate(all_narratives)}
subnarrative_to_index = {sn: i for i, sn in enumerate(all_subnarratives)}

def encode_labels(narratives, subnarratives):
    narrative_vector = [0] * len(all_narratives)
    subnarrative_vector = [0] * len(all_subnarratives)

    for n in narratives:
        narrative_vector[narrative_to_index[n]] = 1
    for sn in subnarratives:
        subnarrative_vector[subnarrative_to_index[sn]] = 1

    return narrative_vector + subnarrative_vector

for article in articles_data:
    article["labels"] = encode_labels(article["narratives"], article["subnarratives"])

# Encode Labels
Encode the labels for each article using the generated vocabularies.

In [ ]:

def encode_labels(narratives, subnarratives):
    narrative_vector = [0] * len(all_narratives)
    subnarrative_vector = [0] * len(all_subnarratives)

    for n in narratives:
        narrative_vector[narrative_to_index[n]] = 1
    for sn in subnarratives:
        subnarrative_vector[subnarrative_to_index[sn]] = 1

    return narrative_vector + subnarrative_vector

for article in articles_data:
    article["labels"] = encode_labels(article["narratives"], article["subnarratives"])

# Create Dataset Class
Define a custom Dataset class to handle the tokenized data and labels.

In [8]:
class NarrativeDataset(Dataset):
    def __init__(self, articles):
        self.articles = articles

    def __len__(self):
        return len(self.articles)

    def __getitem__(self, idx):
        article = self.articles[idx]
        inputs = tokenizer(
            article["text"],
            padding="max_length",
            truncation=True,
            max_length=512,
            return_tensors="pt"
        )
        labels = torch.tensor(article["labels"], dtype=torch.float)
        return {**inputs, "labels": labels}

# Initialize Model
Initialize the RoBERTa model for sequence classification with the appropriate number of labels.

In [ ]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

num_labels = len(all_narratives) + len(all_subnarratives)
model = RobertaForSequenceClassification.from_pretrained(
    "roberta-base", 
    num_labels=num_labels
)
model = model.to(device)
model.train()

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

# Train-Validation Split
Split the data into training and validation sets and create DataLoader instances.

In [ ]:

train_data, val_data = train_test_split(articles_data, test_size=0.2, random_state=42)
train_dataset = NarrativeDataset(train_data)
val_dataset = NarrativeDataset(val_data)

train_loader = DataLoader(train_dataset, batch_size=6, shuffle=True, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=6, shuffle=False, pin_memory=True)

# Training Loop
Define and execute the training loop, including the optimizer and loss calculation.

In [ ]:

optimizer = AdamW(model.parameters(), lr=2e-5)

num_epochs = 3
for epoch in range(num_epochs): 
    print(f"Epoch {epoch + 1}/{num_epochs}")
    epoch_loss = 0
    progress_bar = tqdm.tqdm(train_loader, desc="Processing Batches", leave=True)
    
    for batch in progress_bar:
        optimizer.zero_grad()
        
        inputs = {key: val.squeeze(1).to(device) for key, val in batch.items() if key != "labels"}
        labels = batch["labels"].to(device)
        
        outputs = model(**inputs, labels=labels)
        loss = outputs.loss
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        progress_bar.set_postfix({"Batch Loss": f"{loss.item():.4f}"})
    
    print(f"Epoch {epoch + 1} completed.")
    print(f"- Average Loss: {epoch_loss / len(train_loader):.4f}")
    
   

Epoch 1/3


Processing Batches: 100%|██████████| 27/27 [05:38<00:00, 12.54s/it, Batch Loss=0.3569]


Epoch 1 completed.
- Average Loss: 0.3757
Epoch 2/3


Processing Batches: 100%|██████████| 27/27 [05:07<00:00, 11.39s/it, Batch Loss=0.2439]


Epoch 2 completed.
- Average Loss: 0.2817
Epoch 3/3


Processing Batches: 100%|██████████| 27/27 [04:43<00:00, 10.50s/it, Batch Loss=0.2757]

Epoch 3 completed.
- Average Loss: 0.2243


# Evaluation
Define a function to evaluate the model on the validation set and print the evaluation metrics.

In [ ]:
def get_predictions(model, data_loader, device, threshold=0.5):
    model.eval()
    all_predictions = []
    all_labels = []
    with torch.no_grad():
        for batch in data_loader:
            inputs = {key: val.squeeze(1).to(device) for key, val in batch.items() if key != "labels"}
            labels = batch["labels"].to(device)
            
            outputs = model(**inputs)
            logits = outputs.logits
            probs = torch.sigmoid(logits)  
            preds = (probs > threshold).int() 
            
            all_predictions.append(preds.cpu())
            all_labels.append(labels.cpu())
    
    all_predictions = torch.cat(all_predictions, dim=0)
    all_labels = torch.cat(all_labels, dim=0)
    return all_predictions.numpy(), all_labels.numpy()

def evaluate_model(model, data_loader, device, val_data, class_labels, print_report=True):
    y_pred, y_true = get_predictions(model, data_loader, device)

    hamming = hamming_loss(y_true, y_pred)
    macro_f1 = f1_score(y_true, y_pred, average='macro', zero_division=0)
    micro_f1 = f1_score(y_true, y_pred, average='micro', zero_division=0)
    subset_accuracy = (y_true == y_pred).all(axis=1).mean()

    if print_report:
        report = classification_report(
            y_true, y_pred, target_names=class_labels, digits=2, zero_division=0
        )
        print("\nClassification Report:\n")
        print(report)

    return {"Hamming Loss": hamming, "Macro F1": macro_f1, "Micro F1": micro_f1, "Subset Accuracy": subset_accuracy}

print("Evaluating on Validation Set...")
evaluation_results = evaluate_model(model, val_loader, device, val_data, all_narratives + all_subnarratives)
print(evaluation_results)

Evaluating on Validation Set...

Classification Report:

                                                                                                            precision    recall  f1-score   support

                                                                          CC: Climate change is beneficial       0.00      0.00      0.00         1
                                                                  CC: Controversy about green technologies       0.00      0.00      0.00         3
                                                                         CC: Criticism of climate movement       0.00      0.00      0.00         4
                                                                         CC: Criticism of climate policies       0.00      0.00      0.00         4
                                                             CC: Criticism of institutions and authorities       0.00      0.00      0.00         5
                                                      